In [17]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function newton(F,V,n)
    x = V
    for i = 1:n
        j = ForwardDiff.jacobian(F,x)
        R = inv(j)
        if norm(R*F(x))<10^(-3)
            return x
        end
        x = x-R*F(x)
    end
    return x
end

newton (generic function with 1 method)

In [2]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

orig_is_subset (generic function with 1 method)

In [43]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function krawczyk(F,V,n)
    x = newton(F,V,n)
    @show x
    j = ForwardDiff.jacobian(F,x)
    R = inv(j)
    V2 =  x+norm(R*F(x))*[(-2. .. 2.) for i in 1:size(R)[1]]
    
    j = ForwardDiff.jacobian(F,V2)
    c = mid.(V2)
    R = inv(ForwardDiff.jacobian(F,c)) 
    M = Matrix{Float64}(I,size(R))-R*j
    K = c -R*F(c)+M*(V2-c)
    @show V2
    @show K
    if orig_is_subset(K,V2)
        return K
    else
        return false
    end
end

f((x1,x2)) = x1 + 2*x2 +1
g((x1,x2)) = x1^2 + 2*(x2^2) -1
F((x,y))=[f((x,y));g((x,y))]

X = [1.01, 0.01]

@show krawczyk(F,X,5)



x = [0.33357138648158435, -0.6667856932407922]
V2 = Interval{Float64}[[0.333039, 0.334104], [-0.667318, -0.666253]]
K = Interval{Float64}[[0.333332, 0.333335], [-0.666668, -0.666666]]
krawczyk(F, X, 5) = Interval{Float64}[[0.333332, 0.333335], [-0.666668, -0.666666]]


2-element Vector{Interval{Float64}}:
  [0.333332, 0.333335]
 [-0.666668, -0.666666]